In [229]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm


#불러올 페이지 수 (총 7222페이지)
i = 50
count = 0
df = pd.DataFrame()

for i in tqdm(range(1,(i+1))):
    requestURL = 'https://nedrug.mfds.go.kr/searchDrug?sort=&sortOrder=&searchYn=&ExcelRowdata=&page='+str(i)+'&searchDivision=detail&itemName=&entpName=&ingrName1=&ingrName2=&ingrName3=&itemSeq=&stdrCodeName=&atcCodeName=&indutyClassCode=&sClassNo=&narcoticKindCode=&cancelCode=&etcOtcCode=&makeMaterialGb=&searchConEe=AND&eeDocData=&searchConUd=AND&udDocData=&searchConNb=AND&nbDocData=&startPermitDate=&endPermitDate='
    res = requests.get(requestURL)
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    #print(soup.prettify())
    
    #각 제품의 request보낼 url모으기
    itemSeqList=[]
    links = soup.find_all("a")
    for a in links:
        href = a.attrs['href']
        if "itemSeq" in href:
            #print(href)
            itemSeqList.append(href)
        if len(itemSeqList) == 15:
            break
    #print(itemSeqList)
    
    #한 페이지당 15개 data가 있음
    #data 가져오기
    for j in range(0,len(itemSeqList)):
        cindex = []
        rindex = [str(j+1)]
        
        detailURL='https://nedrug.mfds.go.kr'+str(itemSeqList[j])
        res2 = requests.get(detailURL)
        html_doc2 = res2.text
        soup2 =BeautifulSoup(html_doc2, 'html.parser')
        #print(soup2.prettify())
            
        #col index 가져오기
        FILTER = ['년도','생산실적','순번','변경일자','변경항목'] #여기에 해당되는건 NO
        for title in soup2.find_all('th'):
            if(title.get_text() in FILTER):
                break;
            cindex.append(title.get_text())
        datalist = []
        flag = 0
        for DATA in soup2.find_all('td'):
            #print(DATA.get_text())
            flag += 1
            datalist.append(DATA.get_text())
            if(flag == len(cindex)):
                break
                
        #유효성분 가져오기
        cindex.append("유효성분")
        ingredient = soup2.find_all('h3', class_='cont_title3 mt27 pb10')
        if ingredient:
            for ing in ingredient:
                datalist.append(ing.get_text().replace('유효성분 : ',''))
        else :
            datalist.append("")
            
        #유효성분 분량 가져오기
        #나중에 여기서 a[0]으로 세세히 나눌 수 있을 듯?
        cindex.append("분량")
        amount = soup2.find_all('p', class_='note')
        adic = {}
        for alist in amount:
            temp = alist.get_text()
            a = temp.split('\xa0\xa0|\xa0\xa0')            
            #print(temp)
            if a[0] != '조회 결과가 없습니다.':
                i_name = a[1].replace('성분명 : ','')
                i_amount = a[2].replace('분량 : ',' ')
                i_unit = a[3].replace('단위 : ', '')
                #print(i_name + i_amount + i_unit)
                adic[i_name] = i_amount+i_unit
        if adic == {}:
            datalist.append("")
        else:
            s = ""
            itemList = adic.items()
            for item in itemList:
                s += item[0] + " :" + item[1] + "\n"
            datalist.append(s)
            
        #효능효과 추가하기
        cindex.append("효능효과")
        way = soup2.find_all('div', id ='_ee_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
        #용법용량 추가하기
        cindex.append("용법용량")
        way = soup2.find_all('div', id ='_ud_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
        #주의사항 추가하기
        cindex.append("주의사항")
        way = soup2.find_all('div', id ='_nb_doc')
        s = ""
        for w in way:
            s += w.get_text()
        
        datalist.append(s)
        
            
        #URL 추가하기
        cindex.append("URL")
        datalist.append(detailURL)
        
        #추출 후 남은 데이터
        cindex.append("추출 후 남은 데이터")
        datalist.append("#none")
        
        #데이터 값 수정
        datalist[6] = "kdrug-" + datalist[6]
        for i in range(len(cindex)):
            datalist[i] = datalist[i].replace('제조일로부터 ','')
        #데이터 프레임 만들기
        temp = pd.DataFrame(data= [datalist],index=rindex, columns = cindex)
        
        
        #데이터 프레임 합치기
        df = pd.concat([df, temp])
        
        

df.to_excel("크롤링한 모든 데이터(의약품안전나라).xlsx", index = False )
print("전체 data 가져오기 성공")

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [04:14<00:00,  5.10s/it]


전체 data 가져오기 성공


<h2>필요한 열만 추출하기

In [230]:
finalDf = df[['품목기준코드','제품명','업체명','성상','유효성분','분량','효능효과','용법용량','주의사항','전문/일반','저장방법','사용기간','포장정보','URL','추출 후 남은 데이터']]
finalDf.rename(columns={'품목기준코드':'제품코드','업체명':'제조사'},inplace=True)
finalDf.to_excel("result(의약품안전나라).xlsx", index = False)
print("최종 excel파일 생성 완료")

최종 excel파일 생성 완료
